In [7]:
# Open Drug Discovery Toolkit
import oddt
import oddt.docking
import oddt.interactions
import numpy as np
import pandas as pd

# Докинг

## Для реализации докинга нам потребуется:
- структуры рецептора и лиганда в формате pdbqt
- координаты центра лиганды
- размер ячейки для докинга в Ангстремах
- количество ядер (параметр по умолчанию)
- точность расчетов по Autodock Vina (параметр по умолчанию)
- число конформаций лиганда (по умолчаю данный параметр не может быть более 9)
- максимальное отклонение энергии (параметр по умолчанию)

[Документация к AutodockVina](https://oddt.readthedocs.io/en/latest/rst/oddt.docking.html)

In [2]:
prot = next(oddt.toolkit.readfile('pdbqt','2rh1_prot.pdbqt'))
lig = next(oddt.toolkit.readfile('pdbqt','2rh1_ligand.pdbqt'))

In [3]:
center = [-29.519545, 9.2343645, 6.9440446]

In [4]:
res={}
for i in [-1,0,1]:
    for j in [-1,0,1]:
        for k in [-1,0,1]:
            ckey = '{}_{}_{}'.format(i,j,k)
            dock_obj= oddt.docking.AutodockVina.autodock_vina(
                protein=prot,size=(20,20,20),center=[center[0]+i, center[1]+j, center[2]+k],
                executable='/opt/miniconda3/envs/peptogrid/bin/qvina2.1', autocleanup=True, num_modes=9)

            print(dock_obj.tmp_dir)
            print(" ".join(dock_obj.params))

            res[ckey] = dock_obj.dock([lig],prot)

/tmp/autodock_vina_yzxOS7
--center_x -30.519545 --center_y 8.2343645 --center_z 5.9440446 --size_x 20 --size_y 20 --size_z 20 --exhaustiveness 8 --num_modes 9 --energy_range 3
/tmp/autodock_vina_3tZUtJ
--center_x -30.519545 --center_y 8.2343645 --center_z 6.9440446 --size_x 20 --size_y 20 --size_z 20 --exhaustiveness 8 --num_modes 9 --energy_range 3
/tmp/autodock_vina_nrhNrb
--center_x -30.519545 --center_y 8.2343645 --center_z 7.9440446 --size_x 20 --size_y 20 --size_z 20 --exhaustiveness 8 --num_modes 9 --energy_range 3
/tmp/autodock_vina_mEUEXw
--center_x -30.519545 --center_y 9.2343645 --center_z 5.9440446 --size_x 20 --size_y 20 --size_z 20 --exhaustiveness 8 --num_modes 9 --energy_range 3
/tmp/autodock_vina_eC9UKw
--center_x -30.519545 --center_y 9.2343645 --center_z 6.9440446 --size_x 20 --size_y 20 --size_z 20 --exhaustiveness 8 --num_modes 9 --energy_range 3
/tmp/autodock_vina_1KVjnL
--center_x -30.519545 --center_y 9.2343645 --center_z 7.9440446 --size_x 20 --size_y 20 --size

## Результаты

В результате получили 9 конформаций и их описание для каждого выбранного центра:
- формула лиганда
- степень связи в kcal/mol
- степень отклонения от лучшей конформации
- название первого остатка

Заметим, что результаты отсортированы по лигандам, затем по степени связи - от наилучшей к худшей.

In [5]:
for i,r in enumerate(res['0_0_0']):
    print("%4d%10s%8s%8s%8s" %(i,r.formula, r.data['vina_affinity'],  r.data['vina_rmsd_ub'], r.residues[0].name))

   0 C18H4N2O2    -9.9   0.000     UNL
   1 C18H4N2O2    -9.3   5.629     UNL
   2 C18H4N2O2    -9.2   3.546     UNL
   3 C18H4N2O2    -9.1   6.221     UNL
   4 C18H4N2O2    -8.3   4.120     UNL
   5 C18H4N2O2    -8.2   4.438     UNL
   6 C18H4N2O2    -8.1   6.692     UNL
   7 C18H4N2O2    -8.1   9.214     UNL
   8 C18H4N2O2    -7.9   6.823     UNL


In [8]:
resList = []
for i,r in enumerate(res['0_0_0']):
    hbs = oddt.interactions.hbonds(prot,r)
    stack= oddt.interactions.pi_stacking(prot,r)
    phob = oddt.interactions.hydrophobic_contacts(prot,r)
    resList.append([i, r.formula, r.data['vina_affinity'],  r.data['vina_rmsd_ub'], r.residues[0].name,
               len(hbs[1]), len(stack[1]), len(phob[1])])
resList = sorted(resList, key = lambda i: i[2], reverse = True)
resTable = pd.DataFrame(np.array(resList), columns = ['ID', 'Formula', 'Energy', 'RMSD/ub', 'Resn',
                                                      'Number of hydrogen bonds', 'Number of pi stacking',
                                                      'Number of hydrophobic contacts'])
resTable

,ID,Formula,Energy,RMSD/ub,Resn,Number of hydrogen bonds,Number of pi stacking,Number of hydrophobic contacts
0,0,C18H4N2O2,-9.9,0.000,UNL,6,0,20
1,1,C18H4N2O2,-9.3,5.629,UNL,2,0,22
2,2,C18H4N2O2,-9.2,3.546,UNL,2,0,22
3,3,C18H4N2O2,-9.1,6.221,UNL,1,0,19
4,4,C18H4N2O2,-8.3,4.120,UNL,2,0,16
5,5,C18H4N2O2,-8.2,4.438,UNL,2,0,24
6,6,C18H4N2O2,-8.1,6.692,UNL,3,0,14
7,7,C18H4N2O2,-8.1,9.214,UNL,4,0,11
8,8,C18H4N2O2,-7.9,6.823,UNL,3,0,20


In [9]:
for ckey in res.keys():
    for i,r in enumerate(res[ckey]):
        r.write(filename='pose_{}_{}.pdb'.format(ckey,i), format='pdb')